In [224]:
import pandas as pd
import matplotlib.pyplot as plt
# from utils.Stats import KeyPointsData,AllStats,AggregateStats
import numpy as np
import os
import csv
import warnings
# from utils.Builder import Builder
from utils.Tools import RallyExtraction,split_string,Translate, ServeTranslate
from utils.SeqMining import GetStrokes

In [225]:
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
# matches = pd.read_csv('BaseData/charting-m-matches.csv', encoding='unicode_escape',quoting=csv.QUOTE_NONE)
# points = pd.read_csv('ProjData/Points.csv',encoding='unicode_escape',quoting=csv.QUOTE_NONE)

In [226]:
# Builder("Roger Federer",(2006,2008))

In [227]:
FedererGames = pd.read_csv('FedererStart.csv')
FedererPoints = pd.read_csv('FedererPoints.csv')

In [228]:
# FedererPoints = pd.merge(FedererPoints,FedererGames[['match_id','Round']],how='left',on='match_id')
# FedererPoints = pd.merge(FedererPoints,FedererGames[['match_id','Best of']],how='left',on='match_id')

In [229]:
# FedererPoints

In [230]:
# a = ['S','R','Q','P',' ']
# FedererPoints = FedererPoints[~FedererPoints['1st'].isin(a)]
# FedererPoints = FedererPoints[~FedererPoints['2nd'].isin(a)]


In [231]:
def SnV_Situation(data):
    Situation = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        serve = r[:2]
        if(serve[-1] == "+"):
            Situation.append(1)
        else:
            Situation.append(0)
    
    return Situation

def Net_Situation(data):
    SituationS = []
    SituationR = []
    Rallys = RallyExtraction(data)
    
    for r in Rallys:
        split = split_string(r)
        splitS = split[::2]
        splitR = split[1::2]
        foundS = 0
        for s in splitS:
            if(s.find('+') != -1):
                foundS = 1
                break
            
        if(foundS == 1):
            SituationS.append(1)
        else:
            SituationS.append(0)

        foundR = 0
        for s_r in splitR:
            
            if(s_r.find('+') != -1):
                foundR = 1
                break
            
        if(foundR == 1):
            SituationR.append(1)
        else:
            SituationR.append(0)
    
    return SituationS,SituationR

def Serve_Situation(data):
    Situation = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        if(r[0] == "4"):
            Situation.append("Wide")
        elif(r[0] == "5"):
            Situation.append("Body")
        else:
            Situation.append("Down The T")
    
    return Situation

def RallySize(data):
    Situation = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        size = len(split_string(r))
        if(size == 1):
            Situation.append(size)
        else:
            Situation.append(size-1)

    
    return Situation

def FinalSHOT_Type_N_Direction(data):
    Type = []
    Direction = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        r = ServeTranslate(r)
        print(r)
        if(len(r)<=3):
            Type.append('No Final Shot')
            Direction.append('No Final Shot')
        else:
            Return = split_string(r)[-1]
            print(Return)
            Type.append(Return[0])
            if(len(Return) > 1):
                Direction.append(Return[1])
            else:
                Direction.append('No Direction')

    return Type,Direction

def ReturnType_N_Direction(data):
    Type = []
    Direction = []
    Rallys = RallyExtraction(data)
    for r in Rallys:
        r = ServeTranslate(r)
        if(len(r)<=3):
            Type.append('No Return')
            Direction.append('No Return')
        else:
            Return = split_string(r)[1]
            Type.append(Return[0])
            if(len(Return) > 1):
                Direction.append(Return[1])
            else:
                Direction.append('No Direction')

    return Type,Direction

def PointSize(data):
    data.loc[(data['Rally Size'] <= 3), ['ShortPoint']] = 1
    data.loc[((data['Rally Size'] > 3) & (data['Rally Size'] <=7)) , ['MediumPoint']] = 1
    data.loc[(data['Rally Size'] > 7), ['LargePoint']] = 1


def GameAhead(data):
    Situation = []
    for _,row in data.iterrows():
        split = row['Pts'].split('-')
        if(row['Server'] == True):
            if(split[0] > split[1]):
                Situation.append(1)
            elif(split[0] == split[1]):
                Situation.append(0)
            else:
                Situation.append(-1)
        else:
            if(split[0] > split[1]):
                Situation.append(-1)
            elif(split[0] == split[1]):
                Situation.append(0)
            else:
                Situation.append(1)
    
    return Situation

def SetAhead(data,player):
    data.loc[((data['Player 1'] == player)&(data['Gm1'] > data['Gm2'])), ['isAheadSet']] = 1
    data.loc[((data['Player 1'] == player)&(data['Gm1'] == data['Gm2'])), ['isAheadSet']] = 0
    data.loc[((data['Player 1'] == player)&(data['Gm1'] < data['Gm2'])), ['isAheadSet']] = -1
    data.loc[((data['Player 2'] == player)&(data['Gm1'] > data['Gm2'])), ['isAheadSet']] = -1
    data.loc[((data['Player 2'] == player)&(data['Gm1'] == data['Gm2'])), ['isAheadSet']] = 0
    data.loc[((data['Player 2'] == player)&(data['Gm1'] < data['Gm2'])), ['isAheadSet']] = 1
    
def MatchAhead(data,player):
    data.loc[((data['Player 1'] == player)&(data['Set1'] > data['Set2'])), ['isAheadMatch']] = 1
    data.loc[((data['Player 1'] == player)&(data['Set1'] == data['Set2'])), ['isAheadMatch']] = 0
    data.loc[((data['Player 1'] == player)&(data['Set1'] < data['Set2'])), ['isAheadMatch']] = -1
    data.loc[((data['Player 2'] == player)&(data['Set1'] > data['Set2'])), ['isAheadMatch']] = -1
    data.loc[((data['Player 2'] == player)&(data['Set1'] == data['Set2'])), ['isAheadMatch']] = 0
    data.loc[((data['Player 2'] == player)&(data['Set1'] < data['Set2'])), ['isAheadMatch']] = 1


In [232]:
FedererPoints['SnV'] = SnV_Situation(FedererPoints)
NetServer, NetReceiver = Net_Situation(FedererPoints)
FedererPoints['Net_Server'] = NetServer
FedererPoints['Net_Returner'] = NetReceiver
FedererPoints['Serve'] = Serve_Situation(FedererPoints)
FedererPoints['Rally Size'] = RallySize(FedererPoints)
FedererPoints['ShortPoint'] = 0
FedererPoints['MediumPoint'] = 0
FedererPoints['LargePoint'] = 0
PointSize(FedererPoints)
FedererPoints['isAheadGame'] = GameAhead(FedererPoints)
FedererPoints['isAheadSet'] = 0
FedererPoints['isAheadMatch'] = 0
SetAhead(FedererPoints,'Roger Federer')
MatchAhead(FedererPoints,'Roger Federer')
FedererPoints['DecisiveSet'] = 0
FedererPoints.loc[((FedererPoints['Best of'] == 5) & (FedererPoints['Set#'] == 5)), ['DecisiveSet']] = 1
FedererPoints.loc[((FedererPoints['Best of'] == 3) & (FedererPoints['Set#'] == 3)), ['DecisiveSet']] = 1
FedererPoints['Tiebreaker'] = 0
FedererPoints.loc[((FedererPoints['Gm1'] == 6) & (FedererPoints['Gm2'] == 6)), ['Tiebreaker']] = 1



In [233]:
Type, Direction = ReturnType_N_Direction(FedererPoints)
FedererPoints['Return Type'] = Type
FedererPoints['Return Direction'] = Direction


In [234]:
dic = GetStrokes()

In [235]:
FedererPoints['Return Type'] = FedererPoints['Return Type'].map(dic)
FedererPoints['Return Direction'] = FedererPoints['Return Direction'].map(dic)

In [236]:
strokes = {
    'f' : 'Forehand',
    'b' : 'Backhand',
    'r' : 'FH Slice',
    's' : 'BH Slice',
    'v' : 'FH Volley',
    'z' : 'BH Volley',
    'o' :'standard overhead/smash',
    'p' :'backhand overhead/smash',
    'u' :'forehand drop shot',
    'y' :'backhand drop shot',
    'l' :'forehand lob',
    'm' :'backhand lob',
    'h' :'forehand half-volley',
    'i' :'backhand half-volley',
    'j' :'forehand swinging volley',
    'k' :'backhand swinging volley',
    't' : 'trick shot',
    'q' : 'unknown',
    'e' : 'any'
}

direction = {
    '1' : 'Cruzado',
    '2' : 'Meio',
    '3' : 'Paralelo',
}

columns_svr = []
for value in strokes.values():
    columns_svr.append(value + '_svr')

columns_ret = []
for value in strokes.values():
    columns_ret.append(value + '_ret')

new_dic = {}
value = 0
for key in strokes.keys():
    new_dic[key] = value
    value += 1

# FedererPoints[columns] = 0

In [237]:
for i in range(1,6):
    FedererPoints[f'Last_{i}_pts'] = FedererPoints['Victor'].shift(1).rolling(window=i, min_periods=1).sum().fillna(0).astype(int)

In [238]:
Rallys = RallyExtraction(FedererPoints)
Rallys = [Translate(r) for r in Rallys]

Matrix_svr = np.empty((0,len(new_dic)))
Matrix_ret = np.empty((0,len(new_dic)))

for r in Rallys:
    r_svr = r[1::2][1::2]
    r_ret = r[1::2][::2]
    Pt_svr = np.zeros(len(new_dic))
    Pt_ret = np.zeros(len(new_dic))
    Conteudo_svr = set(r_svr)
    Conteudo_ret = set(r_ret)

    for key,value in new_dic.items():
        if(key in Conteudo_svr):
            Pt_svr[value] = 1
            
    for key,value in new_dic.items():
        if(key in Conteudo_ret):
            Pt_ret[value] = 1
    
    Matrix_svr = np.append(Matrix_svr,[Pt_svr],axis=0)
    Matrix_ret = np.append(Matrix_ret,[Pt_ret],axis=0)

FedererPoints[columns_svr] = Matrix_svr.astype(int)
FedererPoints[columns_ret] = Matrix_ret.astype(int)

In [239]:
Rallys[1][1::2][1::2]

'f'

In [240]:
FedererPoints

,match_id,Pt,Set1,Set2,Set#,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner,Player 1,Player 2,Surface,Victor,Server,GameWinner,SetWinner,Ace,Winner,Unforced Error,Forced Error,Double Fault,Round,Best of,SnV,Net_Server,Net_Returner,Serve,Rally Size,ShortPoint,MediumPoint,LargePoint,isAheadGame,isAheadSet,isAheadMatch,DecisiveSet,Tiebreaker,Return Type,Return Direction,Last_1_pts,Last_2_pts,Last_3_pts,Last_4_pts,Last_5_pts,Forehand_svr,Backhand_svr,FH Slice_svr,BH Slice_svr,FH Volley_svr,BH Volley_svr,standard overhead/smash_svr,backhand overhead/smash_svr,forehand drop shot_svr,backhand drop shot_svr,forehand lob_svr,backhand lob_svr,forehand half-volley_svr,backhand half-volley_svr,forehand swinging volley_svr,backhand swinging volley_svr,trick shot_svr,unknown_svr,any_svr,Forehand_ret,Backhand_ret,FH Slice_ret,BH Slice_ret,FH Volley_ret,BH Volley_ret,standard overhead/smash_ret,backhand overhead/smash_ret,forehand drop shot_ret,backhand drop shot_ret,forehand lob_ret,backhand lob_ret,forehand half-volley_ret,backhand half-volley_ret,forehand swinging volley_ret,backhand swinging volley_ret,trick shot_ret,unknown_ret,any_ret
0,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,1,0,0,1,0.0,0.0,0-0,1.0,True,1,6*,False,False,1,Roger Federer,James Blake,Hard,True,True,True,True,True,False,False,False,False,F,5,0,0,0,Down The T,1,1,0,0,0,0,0,0,0,No Return,No Return,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,2,0,0,1,0.0,0.0,15-0,1.0,True,1,6n,4b28f2n@,False,2,Roger Federer,James Blake,Hard,False,True,True,True,False,False,True,False,False,F,5,0,0,0,Wide,3,1,0,0,1,0,0,0,0,Backhand,Meio,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,3,0,0,1,0.0,0.0,15-15,1.0,True,1,6*,False,Blake challenged incorrectly,1,Roger Federer,James Blake,Hard,True,True,True,True,True,False,False,False,False,F,5,0,0,0,Down The T,1,1,0,0,0,0,0,0,0,No Return,No Return,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,4,0,0,1,0.0,0.0,30-15,1.0,True,1,6d,4b29f3b2f1f3n#,False,1,Roger Federer,James Blake,Hard,True,True,True,True,False,False,False,True,False,F,5,0,0,0,Wide,6,0,1,0,1,0,0,0,0,Backhand,Meio,1,1,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20061119-M-Masters_Cup-F-Roger_Federer-James_B...,5,0,0,1,0.0,0.0,40-15,1.0,True,1,5s28f1f2f2n@,False,False,2,Roger Federer,James Blake,Hard,False,True,True,True,False,False,True,False,False,F,5,0,0,0,Body,5,0,1,0,1,0,0,0,0,BH Slice,Meio,1,2,2,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7717,20060107-M-Doha-F-Roger_Federer-Gael_Monfils,161,1,0,2,6.0,6.0,4-3,22.0,True,2,6n,6f38b2f+1f3*,False,2,Roger Federer,Gael Monfils,Hard,False,False,True,True,False,True,False,False,False,F,3,0,0,1,Down The T,4,0,1,0,-1,0,1,0,1,Forehand,Paralelo,0,1,1,2,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7718,20060107-M-Doha-F-Roger_Federer-Gael_Monfils,162,1,0,2,6.0,6.0,5-3,22.0,True,2,6d,5f28f2f3*,False,1,Roger Federer,Gael Monfils,Hard,True,False,True,True,False,True,False,False,False,F,3,0,0,0,Body,3,1,0,0,-1,0,1,0,1,Forehand,Meio,0,0,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7719,20060107-M-Doha-F-Roger_Federer-Gael_Monfils,163,1,0,2,6.0,6.0,4-5,22.0,True,1,4b2d#,False,False,1,Roger Federer,Gael Monfils,Hard,True,True,True,True,False,False,False,True,False,F,3,0,0,0,Wide,2,1,0

In [ ]:
FedererPoints.to_csv('FedererPoints.csv', index=False)